In [1]:
import dask.dataframe as dd

from dask.diagnostics import ProgressBar

nyc_data = dd.read_parquet("nyc_final")

In [2]:
with ProgressBar():
    
    vehicle_age_by_year = nyc_data["Vehicle Year"].value_counts().compute()

vehicle_age_by_year

[########################################] | 100% Completed | 1.39 sms


0.0       8597125
2013.0    2847241
2014.0    2733114
2015.0    2423991
2012.0    2070721
           ...   
2038.0         16
2034.0         14
2039.0         14
2043.0         13
2035.0          9
Name: Vehicle Year, Length: 100, dtype: int64

In [3]:
with ProgressBar():
    condition = (nyc_data["Vehicle Year"] > 0) & (nyc_data["Vehicle Year"] <= 2018)
    vehicle_age_by_year = nyc_data[condition]["Vehicle Year"].value_counts().compute().sort_index()
vehicle_age_by_year

[########################################] | 100% Completed | 1.65 sms


1970.0        775
1971.0        981
1972.0        971
1973.0        977
1974.0        786
1975.0        949
1976.0        911
1977.0       1276
1978.0       1483
1979.0       1887
1980.0       1996
1981.0       1880
1982.0       2165
1983.0       3277
1984.0       5352
1985.0      10366
1986.0      23731
1987.0      32089
1988.0      69378
1989.0      41676
1990.0      77033
1991.0      29880
1992.0      60798
1993.0      65889
1994.0      93434
1995.0     201647
1996.0     219105
1997.0     372120
1998.0     417387
1999.0     535976
2001.0     857503
2002.0    1002039
2003.0    1155652
2004.0    1346054
2005.0    1470562
2006.0    1600278
2007.0    1801329
2008.0    1528968
2009.0    1210311
2010.0    1362565
2011.0    1683922
2012.0    2070721
2013.0    2847241
2014.0    2733114
2015.0    2423991
2016.0    1280707
2017.0     297496
2018.0       2491
Name: Vehicle Year, dtype: int64

In [4]:
nyc_data_filtered = nyc_data[condition]

nyc_data_filtered["Vehicle Age"] = (nyc_data_filtered["Issue Date"].dt.year-nyc_data_filtered["Vehicle Year"]).astype(int)

nyc_data_with_vehicle_age = nyc_data_filtered[nyc_data_filtered["Vehicle Age"] >= 0]

nyc_data_with_vehicle_age.head()

,Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Temp,Vehicle Age
Citation Issued Month Year,,,,,,,,,,,,,,,,,,,,,
201401,1372499283,XBKU88,NJ,2014-01-25,78,VAN,CHEVR,P,16000,35730,...,ALL,ALL,2004.0,0.0,Uknown,Uknown,PAS,RED,28.6,10
201401,1376981166,95431MD,NY,2014-01-20,14,VAN,ISUZU,P,17650,25390,...,ALL,ALL,2013.0,0.0,Uknown,Uknown,COM,BROWN,28.6,1
201401,1371083952,KRG961,SC,2014-01-25,40,SDN,HONDA,P,16890,11240,...,ALL,ALL,2013.0,0.0,Uknown,Uknown,PAS,GRAY,28.6,1
201401,1371083952,KRG961,SC,2014-01-25,40,SDN,HONDA,P,16890,11240,...,ALL,ALL,2013.0,0.0,Uknown,Uknown,PAS,GRAY,28.6,1
201401,1375617916,61243MA,NY,2014-01-16,46,DELV,MERCU,P,27480,35780,...,ALL,ALL,2006.0,0.0,Uknown,Uknown,COM,BROWN,28.6,8


In [5]:
with ProgressBar():
    files = nyc_data_with_vehicle_age.to_parquet("nyc_data_vehicleAge")

[########################################] | 100% Completed | 144.23 s


In [6]:
with ProgressBar():
    descriptive_state = nyc_data_with_vehicle_age["Vehicle Age"].describe().compute()

descriptive_state.round(2)

[########################################] | 100% Completed | 90.62 s


count    28777416.00
mean            6.74
std             5.66
min             0.00
25%             4.00
50%             8.00
75%            13.00
max            47.00
Name: Vehicle Age, dtype: float64

In [7]:
with ProgressBar():
    nyc_data_by_month = nyc_data.groupby("Citation Issued Month Year")
    citations_per_month = nyc_data_by_month["Summons Number"].count().to_frame().compute()

citations_per_month

[########################################] | 100% Completed | 862.78 ms


,Summons Number
Citation Issued Month Year,
201401,708136
201402,641438
201403,899639
201404,879840
201405,941133
201406,940743
201407,961567
201408,901624
201409,1018933


In [8]:
with ProgressBar():
    N = nyc_data_with_vehicle_age["Vehicle Age"].count()
    p = nyc_data_with_vehicle_age["Plate Type"].unique().count()
brownForsythLeft = dd.compute((N-p)/(p-1))

brownForsythLeft

(14388706.5,)

In [9]:
with ProgressBar():
    citations_and_temps = nyc_data[nyc_data.index.to_series() < 201707] \
        .groupby("Citation Issued Month Year") \
        .agg({"Summons Number": "count", "Temp": "mean"})
    correlation_matrix = citations_and_temps.corr().compute()

correlation_matrix

[########################################] | 100% Completed | 77.38 s


,Summons Number,Temp
Summons Number,1.00000,0.14051
Temp,0.14051,1.00000


In [10]:
nyc_data_with_vehicle_age = dd.read_parquet("nyc_data_vehicleAge")

nyc_data_filtered = nyc_data_with_vehicle_age[nyc_data_with_vehicle_age["Plate Type"].isin(["PAS", "COM"])]

In [11]:
passenger_vehicles = nyc_data_filtered[nyc_data_filtered["Plate Type"] == "PAS"]
commercial_vehicles = nyc_data_filtered[nyc_data_filtered["Plate Type"] == "COM"]

median_PAS, median_COM = (
   passenger_vehicles["Vehicle Age"].quantile(0.5),
   commercial_vehicles["Vehicle Age"].quantile(0.5))

In [12]:
condition = (nyc_data_filtered["Plate Type"] == "PAS").astype(int)
median = median_PAS*condition+median_COM*(1-condition)
absolute_deviation = abs(nyc_data_filtered["Vehicle Age"] - median)
nyc_data_filtered["Median Difference"] = absolute_deviation
#display(median.head())

#absolute_deviation.head()

In [13]:
with ProgressBar():
    group_means = nyc_data_filtered.groupby("Plate Type")["Median Difference"].mean().compute()

group_means


[########################################] | 100% Completed | 84.64 s


Plate Type
COM    4.808030
PAS    4.926354
Name: Median Difference, dtype: float64

In [14]:
condition = (nyc_data_filtered["Plate Type"] == "PAS").astype(int)
mean = group_means["PAS"]*condition+group_means["COM"]*(1-condition)
#mean = group_means[nyc_data_filtered["Plate Type"]]
group_mean_variances = (nyc_data_filtered["Vehicle Age"] - mean)**2
nyc_data_filtered["Group Mean Variance"] = group_mean_variances

    #display(group_mean_variances.head())
#display(median.head())
   

In [15]:
with ProgressBar():
    brown_forsythe_right_denominator = nyc_data_filtered["Group Mean Variance"].sum()
    grand_mean = nyc_data_filtered["Median Difference"].mean().compute()

[########################################] | 100% Completed | 85.39 s


In [16]:
brown_forsythe_aggregation = dd.Aggregation(
    "Brown_Forsythe",
    lambda chunk: (chunk.count(), chunk.sum()),
    lambda chunk_count, chunk_sum: (chunk_count.sum(), chunk_sum.sum()),
    lambda group_count, group_sum: group_count*(((group_sum/group_count)-grand_mean)**2))

In [17]:
with ProgressBar():
    group_variances = nyc_data_filtered \
        .groupby("Plate Type") \
        .agg({"Median Difference": brown_forsythe_aggregation}) \
        .compute()

[########################################] | 100% Completed | 86.29 s


In [18]:
brown_forsythe_right_numerator = group_variances.sum()[0]

In [20]:
F_statistic = brownForsythLeft[0] * (brown_forsythe_right_numerator/brown_forsythe_right_denominator)
F_statistic

1036.0283639715024